# arag.ai<br>
Active RAG based LLM Model<br>
Built so that it's dependent on you. Hence works only for you.<br>
Created with ❤️<br><br>
Flow - 
1. Toggle for RAG and Generic LLM<br>if RAG: <br>
2. get the link to docs for embedding 
3. query 
4. result generation <br>else: <br>
2. query 
3. result generation

In [ ]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()
token = os.getenv('TOKEN')
print(token)
API_URL = "https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3-8B-Instruct"
headers = {"Authorization": f"Bearer {token}"}
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.content
data = query({"inputs":"What is Task Decomposition"})
data_str = data.decode('utf-8')  # Decode bytes to string
response_json = json.loads(data_str)  # Parse the JSON string

print("\n".join(response_json[0]['generated_text'].split('\n')[1:]))

In [ ]:
import requests
import json
API_URL = "https://api-inference.huggingface.co/models/google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"
token = os.getenv('TOKEN')
headers = {"Authorization": f"Bearer {token}"}
# def query(payload):
#     response = requests.post(API_URL, headers=headers, json=payload)
#     return response.content
# data = query({"question","Who is the best player in football?"})
# data_str = data.decode('utf-8')  # Decode bytes to string
# response_json = json.loads(data_str)  # Parse the JSON string



import requests

API_URL = "https://api-inference.huggingface.co/models/google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()


prompt = "What's the capital of India"
formatted_docs = "New Delhi is captial of India"
output = query({"inputs": {"question": f"{prompt}","context": f"{formatted_docs})"}})
print(output['answer'])
# print(response_json)#[0]["generated_text"].split('\n'))

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
token = os.getenv('TOKEN')
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-large-uncased-whole-word-masking-finetuned-squad", token=token)
model = AutoModelForCausalLM.from_pretrained("google-bert/bert-large-uncased-whole-word-masking-finetuned-squad", token=token)
generator = pipeline('question-answering', model=model, tokenizer=tokenizer)
prompt = "capital of india?"

In [ ]:
prompt = {
	"question": "What is my name?",
	"context": "My name is Clara and I live in Berkeley."
}
generated_text = generator(prompt)
print(generated_text)


In [ ]:
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
import bs4

# Load a compatible model from sentence-transformers
model = SentenceTransformer("Snowflake/snowflake-arctic-embed-m")

# Wrapper class for embeddings
class EmbeddingsWrapper:
    def __init__(self, model):
        self.model = model

    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_tensor=True).tolist()

    def embed_query(self, text):
        return self.model.encode([text], convert_to_tensor=True).tolist()[0]

# Instantiate the wrapper
embedding_wrapper = EmbeddingsWrapper(model)

# Load Docs
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=('post-content', 'post-title', 'post-header')
        )
    )
)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed and create vectorstore
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_wrapper)
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Retrieve documents and format them
retrieved_docs = retriever.get_relevant_documents("What is Task Decomposition?")
formatted_docs = format_docs(retrieved_docs)



In [ ]:
import requests

API_URL = "https://api-inference.huggingface.co/models/valhalla/bart-large-finetuned-squadv1"
def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()


prompt = "Summarize"
output = query({"inputs": {"question": f"{prompt}","context": f"{formatted_docs})"}})
print(output)

In [ ]:
print(formatted_docs)